<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/DeepLearningFiles/Time_series_and_encoder_decoder_implementations_DL_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Build and create ConvNet and RNN models for image and text data classifications from the chollat's book
import statsmodels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,GRU
from statsmodels.datasets import get_rdataset

In [37]:
# import dataset
data = get_rdataset("AirPassengers", "datasets").data
data

,time,value
0,1949.000000,112
1,1949.083333,118
2,1949.166667,132
3,1949.250000,129
4,1949.333333,121
...,...,...
139,1960.583333,606
140,1960.666667,508
141,1960.750000,461
142,1960.833333,390


In [38]:
data.index = pd.to_datetime(data["time"])
data.drop(columns="time", inplace=True)

In [39]:
data.head()

,value
time,
1970-01-01 00:00:00.000001949,112
1970-01-01 00:00:00.000001949,118
1970-01-01 00:00:00.000001949,132
1970-01-01 00:00:00.000001949,129
1970-01-01 00:00:00.000001949,121


In [40]:
# normalize the data
data = (data - data.mean())/data.std()

In [41]:
# convert the 1D data into sequential data
def create_seq(data, seq_len):
    x = []
    y = []
    for i in range(len(data) - seq_len):
      x.append(data[i:(i+seq_len)])
      y.append(data[i+seq_len])
    return np.array(x), np.array(y)

In [42]:
seq_len = 14
X, y = create_seq(data.values, seq_len)

In [43]:
split = int(0.8*len(data))

In [44]:
# train and test split
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [45]:
# design the model
model = Sequential()
model.add(LSTM(64, return_sequences=True,activation="tanh",input_shape=(seq_len, 1)))
model.add(GRU(64, activation="tanh"))
model.add(Dense(1, activation="linear"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [46]:
# compile the model
model.compile(optimizer="adam", loss="mse")

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 204ms/step - loss: 0.4986 - val_loss: 0.9663
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.2123 - val_loss: 0.4037
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.1838 - val_loss: 0.5386
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.1718 - val_loss: 0.4485
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.1611 - val_loss: 0.3881
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.1625 - val_loss: 0.4000
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1671 - val_loss: 0.3855
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.1526 - val_loss: 0.3633
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1328 - val_loss: 0.3532
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.1381 - val_loss: 0.3420
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.1242 - val_loss: 0.3368
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.1260 - val_l

In [48]:
test_loss = model.evaluate(X_test,y_test)
print("Test loss of the function:",test_loss)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.2600
Test loss of the function: 0.2599867284297943
